In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from fastai.tabular.all import * 
from mock import Mock
from tabnet.utils import *
from tabnet.model import *

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
class LinDecoder(Module):
    def __init__(self, n_cat, n_cont, n_d, ps=0.1, **kwargs):
        store_attr()
        
        self.decoder = nn.Sequential(
            LinBnDrop(n_d, 256, p=ps, act=Mish()),
            LinBnDrop(256, 512, p=ps, act=Mish()),
            LinBnDrop(512, 1024, p=ps, act=Mish())
        )
        
        self.decoder_cont = nn.Sequential(
            LinBnDrop(1024, n_cont, p=ps, bn=False, act=None),
        )
        
        self.decoder_cat = LinBnDrop(1024, n_cat, p=ps, bn=False, act=None)

        
    def forward(self, x):
        xs = x.sum(dim=1)
        decoded = self.decoder(xs)
        decoded_cats = self.decoder_cat(decoded)
        decoded_conts = self.decoder_cont(decoded)

        return torch.cat([decoded_cats, decoded_conts], dim=1)

In [5]:
def _create_shared_blocks(n_in, n_out, n_shared):
    return [_initial_block(n_in, n_out)] + \
            [_rest_block(n_out) for _ in range(n_shared-1)]

def _initial_block(n_in, n_out):
    return nn.Linear(n_in, 2*n_out, bias=False)

def _rest_block(n):
    return nn.Linear(n, 2*n, bias=False)


In [6]:
class TabNetDec2(TabNetBase):
    def __init__(self, n_cat, n_cont, n_d, n_a, n_shared_ft_blocks, n_dec_steps, **kwargs):
        store_attr()
        super().__init__(n_d=n_d, n_a=n_a, n_shared_ft_blocks=n_shared_ft_blocks, **kwargs)
        
        shared_ft_blocks = _create_shared_blocks(self.n_d, self.n_d + self.n_a, self.n_shared_ft_blocks)
        
        self.steps = nn.ModuleList([
                            nn.Sequential(
                                self._create_feature_transform(shared_ft_blocks),
                                nn.Linear(self.n_d+self.n_a, n_cat+n_cont)) for _ in range(self.n_dec_steps)
                        ])
        
        
        
        
    def forward(self, x):
        xs = x.sum(dim=1)
        
        output = 0 
        
        for step in self.steps:
            output = output + step(xs)
        
        return output

In [7]:
class MRL1(Module):
    def __init__(self, lambda_reg=1e-4, eps=1e-5): store_attr()
    
    def forward(self, preds, targ):
        preds, targ = preds*(1-self.S), targ*(1-self.S)
        norm = (targ - targ.mean(dim=0)).pow(2).sum(dim=0).sqrt()
        norm_mask = norm >= 1e-6
        norm = norm[norm_mask]
        error = (preds - targ)
        error = error[:,norm_mask]

        loss = (error / norm).pow(2).sum(dim=1).mean()
        return loss

In [8]:
class MRL2(Module):
    def __init__(self, lambda_reg=1e-4, eps=1e-5): store_attr()
    
    def forward(self, preds, targ):
        preds, targ = preds*(1-self.S), targ*(1-self.S)
        norm = (targ - targ.mean(dim=0)).pow(2).sum(dim=0).sqrt()
        error = (preds - targ)

        loss = (error / norm).abs().sum(dim=1).mean()
        return loss

# Tests

### SS Forest

In [9]:
data_dir = Path('./data')

In [10]:
def extract_gzip(file, dest=None):
    import gzip
    dest = dest or Path(dest)
    with gzip.open(file, 'rb') as f_in:
        with open(dest / file.stem, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [11]:
forest_type_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
forest_path = untar_data(forest_type_url, dest=data_dir, extract_func=extract_gzip)

In [12]:
target = "Covertype"

cat_names = [
    "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3",
    "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4",
    "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9",
    "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14",
    "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19",
    "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24",
    "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29",
    "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34",
    "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39",
    "Soil_Type40"
]

cont_names = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

feature_columns = (
    cont_names + cat_names + [target])

params = dict(cont_names = cont_names, y_names = target, cat_names = cat_names)
procs=[Categorify, FillMissing, Normalize]
model_params = dict(n_d=64, n_a=64, n_steps=5, virtual_batch_size=512, gamma=1.5, bs=1024*16,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=2,
                    n_dec_steps=10, p=0.8)

In [13]:
df = pd.read_csv(forest_path, header=None, names=feature_columns).sample(n=200_000)
df.shape

(200000, 55)

In [14]:
val_pct = 0.2
curriculum = False

In [15]:
to = tabular_pandas(df, **params, 
                    tabular_type=TabularPandasIdentity, val_pct=val_pct)
loss_func = MRL2()

#head = lambda n_cat, n_cont, **kwargs: LinDecoder(n_cat, n_cont, **kwargs)
head = lambda n_cat, n_cont, **kwargs: TabNetDec2(n_cat, n_cont, **kwargs)

In [17]:
vals = []
for val_pct in sorted([0.8, 0.9, 0.99]*3):
    print(val_pct)
    (before, after) = score_before_after_ss(df, params, val_pct=val_pct, decoder_head=head, 
                        loss_func=loss_func, cycle_lr=[(250, slice(5e-3, 1e-1)), (30, slice(1e-3, 1e-1/2)),
                                                       (250, slice(5e-3, 1e-1))], **model_params)
    vals.append((before.item(), after.item(), val_pct))
    res = pd.DataFrame(vals, columns=['before','after','val'])
    res.to_csv('forest_res.csv')

0.8
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


epoch,train_loss,valid_loss,accuracy,time
0,4.139518,3.737472,0.076131,00:01
1,3.725219,3.341648,0.313619,00:01
2,3.334324,2.997022,0.429888,00:01
3,2.977854,2.546309,0.476969,00:01
4,2.638077,1.815441,0.495012,00:01
5,2.358522,1.351173,0.487125,00:01
6,2.141742,1.252292,0.481375,00:01
7,1.972914,1.241264,0.489013,00:01
8,1.838879,1.235695,0.489038,00:01
9,1.730273,1.204323,0.488831,00:01


epoch,train_loss,valid_loss,mse,time
0,0.433419,0.224169,0.264376,00:06
1,0.327559,0.343282,4.049806,00:06
2,0.260606,0.181392,0.814618,00:06
3,0.236425,0.201597,1.787496,00:06
4,0.218239,0.210605,6.216951,00:06
5,0.207673,0.136232,0.317053,00:06
6,0.201271,0.370284,71.247108,00:06
7,0.192511,0.166269,1.196614,00:06
8,0.185730,0.163580,0.428983,00:06
9,0.180510,0.166755,0.441369,00:06


epoch,train_loss,valid_loss,accuracy,time
0,4.061196,3.542257,0.267550,00:01
1,3.757657,1.895187,0.447587,00:01
2,3.454114,1.969438,0.367231,00:01
3,3.145954,1.924480,0.363963,00:01
4,2.839163,1.828862,0.414181,00:01
5,2.565533,1.648348,0.472731,00:01
6,2.342066,1.881508,0.425687,00:01
7,2.166826,1.656682,0.433781,00:01
8,2.029410,1.534977,0.477206,00:01
9,1.915222,1.474438,0.481969,00:01


0.8
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


...

epoch,train_loss,valid_loss,accuracy,time
0,3.724135,3.172584,0.100071,00:01
1,3.365587,1.969381,0.489798,00:01
2,3.033286,1.856953,0.483136,00:01
3,2.741342,2.569924,0.489126,00:01
4,2.497225,1.776818,0.488061,00:01
5,2.300479,1.751820,0.488439,00:01
6,2.130813,2.227268,0.488152,00:01
7,1.994123,1.698404,0.487338,00:01
8,1.877038,1.542839,0.493308,00:01
9,1.780722,1.333005,0.484061,00:01


0.99
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


epoch,train_loss,valid_loss,accuracy,time
0,3.896450,3.795010,0.478020,00:01
1,3.560219,3.014980,0.478919,00:01
2,3.214176,2.417084,0.487859,00:01
3,2.895096,2.000299,0.478712,00:01
4,2.612025,1.744881,0.436086,00:01
5,2.375031,1.503954,0.478606,00:01
6,2.193384,1.339937,0.468788,00:01
7,2.051649,1.286672,0.442298,00:01
8,1.931222,1.244841,0.443308,00:01
9,1.831763,1.251488,0.452455,00:01


epoch,train_loss,valid_loss,mse,time
0,0.474374,0.191109,0.261088,00:07
1,0.340417,0.151098,0.428898,00:06
2,0.275203,0.195647,1.137054,00:06
3,0.251974,0.226837,2.023103,00:06
4,0.231769,0.160876,0.301772,00:06
5,0.211401,0.173226,1.950643,00:06
6,0.211140,0.183071,0.539062,00:06
7,0.205109,0.180771,0.946773,00:06
8,0.198333,0.160334,0.702493,00:06
9,0.191213,0.194855,0.374489,00:06


epoch,train_loss,valid_loss,accuracy,time
0,3.724545,3.996199,0.033520,00:01
1,3.381918,1.896366,0.418152,00:01
2,3.024107,1.802779,0.476490,00:01
3,2.705990,1.755567,0.486667,00:01
4,2.433202,2.122566,0.488071,00:01
5,2.216664,2.166627,0.488162,00:01
6,2.040185,1.874195,0.487783,00:01
7,1.904560,1.676913,0.476884,00:01
8,1.796764,1.510167,0.478581,00:01
9,1.707939,1.385555,0.451146,00:01


In [ ]:
vals = []
for val_pct in sorted([0.2, 0.4, 0.6]*3):
    print(val_pct)
    (before, after) = score_before_after_ss(df, params, val_pct=val_pct, decoder_head=head, 
                        loss_func=loss_func, cycle_lr=[(350, slice(5e-3, 1e-1)), (100, slice(1e-3, 1e-1/2)),
                                                       (350, slice(5e-3, 1e-1))], **model_params)
    vals.append((before.item(), after.item(), val_pct))
    res = pd.DataFrame(vals, columns=['before','after','val'])
    res.to_csv('forest_res_06.csv')

0.2
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


epoch,train_loss,valid_loss,accuracy,time
0,2.542612,2.047883,0.481750,00:03
1,1.723152,1.324158,0.488000,00:03
2,1.411138,1.289773,0.488000,00:03
3,1.243166,1.210041,0.487925,00:03
4,1.132474,1.132369,0.510275,00:03
5,1.053638,1.080519,0.546900,00:03
6,0.992569,0.985550,0.586050,00:03
7,0.942920,0.889092,0.607125,00:03
8,0.902542,0.789889,0.643150,00:03
9,0.867982,0.757093,0.666575,00:03


epoch,train_loss,valid_loss,mse,time
0,0.425444,0.192531,0.259698,00:06
1,0.296967,0.184311,0.274223,00:06
2,0.242917,0.165568,0.628115,00:06
3,0.210719,0.196862,0.365140,00:06
4,0.189669,0.142374,0.477136,00:06
5,0.171603,0.123528,0.311544,00:06
6,0.161670,0.120942,0.385832,00:06
7,0.152687,0.156968,1.675259,00:06
8,0.147407,0.150677,0.954363,00:06
9,0.145116,0.201453,2.610399,00:06


epoch,train_loss,valid_loss,accuracy,time
0,2.893256,1.534404,0.489175,00:03
1,2.060000,1.247031,0.482800,00:03
2,1.676297,1.211532,0.451300,00:03
3,1.462592,1.113515,0.479275,00:03
4,1.327069,1.312574,0.478175,00:03
5,1.231087,1.325644,0.490625,00:03
6,1.158819,1.888572,0.357275,00:03
7,1.102047,1.890496,0.325500,00:03
8,1.057727,1.356754,0.480600,00:03
9,1.021686,1.721212,0.478250,00:03


0.2
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


...

epoch,train_loss,valid_loss,mse,time
0,0.431010,0.366943,0.425780,00:06
1,0.314404,0.126471,0.236889,00:06
2,0.253264,0.139644,0.247976,00:06
3,0.215529,0.090911,0.228810,00:06
4,0.191207,0.104359,0.226914,00:06
5,0.172839,0.121330,0.267857,00:06
6,0.160866,0.109259,0.269155,00:06
7,0.155842,0.133652,0.359030,00:06
8,0.150346,0.165643,0.820758,00:06
9,0.145975,0.130570,0.434500,00:06


epoch,train_loss,valid_loss,accuracy,time
0,3.595074,3.894434,0.487783,00:02
1,3.008435,2.317753,0.488008,00:02
2,2.511711,4.921638,0.488008,00:02
3,2.175171,1.844493,0.488108,00:02
4,1.953831,1.867211,0.492008,00:02
5,1.802418,2.667583,0.513133,00:02
6,1.694982,1.371579,0.399458,00:02
7,1.613646,1.467972,0.492158,00:02
8,1.548957,1.490534,0.502983,00:02
9,1.494435,1.491483,0.484942,00:02


0.6
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8}


epoch,train_loss,valid_loss,accuracy,time
0,3.572642,3.470832,0.478025,00:02
1,2.818635,1.862957,0.508650,00:02
2,2.239708,1.311928,0.546108,00:02
3,1.883254,1.117377,0.592558,00:02
4,1.648524,1.042188,0.622225,00:02
5,1.482383,0.971415,0.632175,00:02
6,1.358013,1.056541,0.606983,00:02
7,1.261214,1.041730,0.608242,00:02
8,1.184683,0.960777,0.634175,00:02
9,1.124302,0.992899,0.627767,00:02


epoch,train_loss,valid_loss,mse,time
0,0.487357,0.292296,0.349379,00:06
1,0.335233,0.161421,0.234944,00:06
2,0.275791,0.113262,0.237314,00:06
3,0.232450,0.122289,0.229871,00:06
4,0.203320,0.098880,0.230970,00:06
5,0.183196,0.112110,0.236135,00:06
6,0.168965,0.122917,0.249181,00:06
7,0.159593,0.133535,0.269581,00:06
8,0.157052,0.212621,0.646718,00:06
9,0.155405,0.153611,0.381058,00:06


In [ ]:
dls = to.dataloaders(bs=model_params['bs'])
dls.n_inp = 2
cbs = [SetPrior(), TabularMasking(p=model_params['p'], curriculum=curriculum),
       MaskRegularizer(model_params['lambda_sparse'])]
model = TabNetSelfSupervised(head, to, **model_params)

In [ ]:
learn = Learner(dls, model, cbs=cbs, loss_func=loss_func, metrics=[mse])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(30, slice(1e-3, 1e-1/2))

In [ ]:
learn.recorder.plot_loss()

In [ ]:
dl = learn.dls.test_dl(df)
preds, targs = learn.get_preds(dl=dl)
mse(preds, targs)

In [ ]:
for p, t in zip(preds[0].tolist(), targs[0].tolist()):
    print(p, t)

In [ ]:
mp = {**model_params, 'virtual_batch_size':100}

In [ ]:
l = tabnet_df_classifier(df, **params, enc=learn.model.enc, val_pct=0.99, **mp)

In [ ]:
l.dls.train.bs = l.dls.train.n//2 if l.dls.train.n < model_params['bs'] else model_params['bs']

In [ ]:
l.lr_find()

In [ ]:
l.fit_one_cycle(100, slice(5e-3, 1e-1))

In [ ]:
l.recorder.plot_loss()

### SS Flowers

In [ ]:
data = load_iris()
X,y = data['data'], data['target']
cont_names = ['s_len', 's_wid', 'p_len', 'p_wid']
cat_names = []
y_names = 'target'
df = pd.concat([pd.DataFrame(X, columns=cont_names),
           pd.DataFrame(y, columns=[y_names])], axis=1)

In [ ]:
model_params = dict(n_d=16, n_a=16, n_steps=3, virtual_batch_size=4, gamma=1.5, bs=32,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=0, p=0.3)
val_pct = 0.2
curriculum = False
head = lambda n_out, **kwargs: LinDecoder(n_out, **kwargs)
loss_func = MSELossFlat()

In [ ]:
to = tabular_pandas(df, cat_names, cont_names, y_names, tabular_type=TabularPandasIdentity, val_pct=val_pct)
dls = to.dataloaders(bs=model_params['bs'])
dls.n_inp = 2
cbs = [SetPrior(), TabularMasking(p=model_params['p'], curriculum=curriculum),
       MaskRegularizer(model_params['lambda_sparse'])]
model = TabNetSelfSupervised(head, to, **model_params)

In [ ]:
learn = Learner(dls, model, cbs=cbs, loss_func=loss_func, metrics=[mse])

In [ ]:
learn.dls.cpu()

In [ ]:
learn.fit_one_cycle(30, slice(1e-4, 1e-3))

In [ ]:
dl = learn.dls.test_dl(df)
preds, targs = learn.get_preds(dl=dl)

In [ ]:
preds[0][:10], targs[0][:10]

### BaseLine

In [ ]:
splits = RandomSplitter()(range_of(df))
dls = TabularPandas(df, cat_names=[], cont_names=['s_len', 's_wid', 'p_len', 'p_wid'], splits=splits,
              procs=[Categorify, FillMissing, Normalize], y_names='target', y_block=CategoryBlock()).dataloaders(bs=4)

In [ ]:
learn = tabular_learner(dls, metrics=[accuracy])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 1e-2)

### TabNet

In [ ]:
model_params = dict(n_d=16, n_a=16, n_steps=5, virtual_batch_size=512, gamma=1.5, bs=30,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=2)

In [ ]:
learn = tabnet_df_classifier(df, cat_names=[], cont_names=['s_len', 's_wid', 'p_len', 'p_wid'],
                                                     y_names='target', val_pct=0.2, **model_params)

In [ ]:
learn.summary()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10, 1e-2)

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()